# Создание модели и конвертация в ONNX

In [7]:
import torch
import segmentation_models_pytorch as smp
import timm

In [8]:
# model = timm.create_model(
#         'mobilenetv3_large_100',
#         num_classes=10,
#         in_chans=3,
#         pretrained=True,
#         exportable=True,
#     )

In [9]:
model = smp.create_model(
    'unet',
    'mobilenet_v2',
    encoder_depth=4,
    decoder_channels=(128, 64, 32, 16),
    classes=11
)   
model = model.half()

In [10]:
model.eval()
model.to('cuda')

test_image = torch.randn(1, 3, 384 , 384, requires_grad=False).half().to('cuda')
orig_out = model(test_image)
orig_out.shape

torch.Size([1, 11, 384, 384])

In [11]:
torch_out = torch.onnx._export(
        model,
        test_image,
        'model.onnx',
        training=torch.onnx.TrainingMode.EVAL,
        export_params=True,
        input_names=["input0"],
        output_names=["output0"],
        operator_export_type=torch.onnx.OperatorExportTypes.ONNX
    )

/tmp/ipykernel_60536/2706819781.py:1: FutureWarning: 'torch.onnx._export' is deprecated in version 1.12.0 and will be removed in 2.0. Please use `torch.onnx.export` instead.
  torch_out = torch.onnx._export(


# Проверка работоспособности модели

In [13]:
import onnx

onnx_model = onnx.load('model.onnx')
onnx.checker.check_model(onnx_model, full_check=True)

def onnx_forward(onnx_file, example_input):
    import onnxruntime

    sess_options = onnxruntime.SessionOptions()
    session = onnxruntime.InferenceSession(onnx_file, providers=['TensorrtExecutionProvider'])
    input_name = session.get_inputs()[0].name
    output = session.run([], {input_name: example_input.cpu().numpy()})
    output = output[0]
    return output

import numpy as np
onnx_out = onnx_forward('model.onnx', test_image)
np.testing.assert_almost_equal(torch_out.data.cpu().numpy(), onnx_out, decimal=1)
np.testing.assert_almost_equal(orig_out.data.cpu().numpy(), torch_out.data.cpu().numpy(), decimal=1)

/home/arseniyzemerov/.local/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'TensorrtExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
